In [1]:
###########################
# Lab 2
###########################

In [2]:
# Train model (feed-forward)

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [3]:
def get_MNIST_train():
    
    mnist_train_data = np.zeros([60000,784]) # fiecare linie este o img 28x28
    mnist_train_labels = np.zeros(60000)
    
    # ~ url = 'https://github.com/golbin/TensorFlow-MNIST/tree/master/mnist/data/train-images-idx3-ubyte.gz'
    # ~ r = requests.get(url, allow_redirects=True)
    # ~ open('train-images-idx3-ubyte.gz', 'wb').write(r.content)
    
    # ~ url = 'https://github.com/golbin/TensorFlow-MNIST/tree/master/mnist/datatrain-images-idx3-ubyte.gz'
    # ~ r = requests.get(url, allow_redirects=True)
    # ~ open('train-labels.idx1-ubyte.gz', 'wb').write(r.content)
    
    f = open('../../t10k-images.idx3-ubyte','r',encoding = 'latin-1')   
    g = open('../../t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_train_data = np.fromfile(f,dtype=np.uint8).reshape(60000,784)
    mnist_train_labels = np.fromfile(g,dtype=np.uint8)
        
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_train_data = mnist_train_data.astype(np.float32)
    mnist_train_labels = mnist_train_labels.astype(np.int64)
        
    return mnist_train_data, mnist_train_labels

In [4]:
def get_MNIST_test():
    
    mnist_test_data = np.zeros([10000,784])
    mnist_test_labels = np.zeros(10000)
    
    f = open('../../t10k-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('../../t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_test_data = np.fromfile(f,dtype=np.uint8).reshape(10000,784)
    mnist_test_labels = np.fromfile(g,dtype=np.uint8)
    
    # Conversii pentru a se potrivi cu procesul de testare    
    mnist_test_data = mnist_test_data.astype(np.float32)
    mnist_test_labels = mnist_test_labels.astype(np.int64)        
    
    return mnist_test_data, mnist_test_labels

In [5]:
import torch.nn as nn

class Retea_MLP(nn.Module):
    
    def __init__(self, nr_neuroni_input, nr_neuroni_hidden, nr_clase):
        
        # Pentru a putea folosi mai departe reteaua, este recomandata mostenirea
        # clasei de baza nn.Module
        super(Retea_MLP,self).__init__()
        
        # Definirea ponderilor si a deplasamentelor din stratul ascuns
        self.w_h = torch.randn(nr_neuroni_input, nr_neuroni_hidden, dtype = torch.float, requires_grad=True)
        self.b_h = torch.randn(nr_neuroni_hidden, dtype = torch.float, requires_grad=True)
        
        # Definirea ponderilor si a deplasamentelor din stratul de iesire
        self.w_o = torch.randn(nr_neuroni_hidden, nr_clase, dtype = torch.float, requires_grad=True)
        self.b_o = torch.randn(nr_clase, dtype = torch.float, requires_grad=True)

        self.relu = nn.ReLU()
        
    
    # Se aduna toate variabilele antrenabile intr-o lista, pentru a putea face referire rapida la ele
    def parameters(self):
        return [self.w_h, self.b_h, self.w_o, self.b_o]
    
    def forward(self,input_batch):
        # Intr-un MLP, intrarea este sub forma unui vector, deci un batch
        # este o matrice de dimensiunea nr_esantioane_batch x dimensiune esantion
        input_batch = torch.from_numpy(input_batch)
        self.hidden = torch.mm(input_batch, self.w_h) + self.b_h

        self.hidden = self.relu(self.hidden)
        
        out = torch.mm(self.hidden, self.w_o) + self.b_o

        # out = softmax(out) - se poate si fara (softmax este inclus in CrossEntropyLoss)
        
        return out

In [6]:
# Instantiem reteaua
mlp = Retea_MLP(28*28,1000,10)

# Specificarea functiei loss
loss_function = nn.CrossEntropyLoss(reduction='sum')

# Specificarea optimizatorului
optim = torch.optim.SGD(mlp.parameters(), lr=1e-5)

In [7]:
# Bucla de invatare
import numpy as np

train_data, train_labels = get_MNIST_train()
batch_size = 128 # Se poate si mai mult in cazul curent, dar este o valoare frecventa
nr_epoci = 15
nr_iteratii = train_data.shape[0] // batch_size # Din simplitate, vom ignora ultimul batch, daca este incomplet


for ep in range(nr_epoci):
    predictii = []
    etichete = []

    for it in range(nr_iteratii):
        # Luam urmatoarele <batch_size> esantioane si etichete
        batch_data = train_data[it*batch_size : it*batch_size+batch_size, :]
        batch_labels = train_labels[it*batch_size : it*batch_size+batch_size]
        # Se calculeaza predictia retelei pentru datele curente (forward pass/ propagare inainte)
        current_predict = mlp.forward(batch_data)

        # Se calculeaza valoarea momentana a functiei loss
        loss = loss_function(current_predict, torch.from_numpy(batch_labels)) 
        
        # Se memoreaza predictiile si etichetele aferente batch-ului actual (pentru calculul acuratetii)
        current_predict = np.argmax(current_predict.detach().numpy(), axis=1)
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,batch_labels))
        
        # Antrenarea propriu-zisa
        
            # 1. Se sterg toti gradientii calculati anteriori, pentru toate variabilele antrenabile
            # deoarece, metoda <backward> acumuleaza noile valori, in loc sa le inlocuiasca.
        optim.zero_grad()
            # 2. Calculul tuturor gradientilor. Backpropagation
        loss.backward()
            # 3. Actualizarea tuturor ponderilor, pe baza gradientilor.
        optim.step()
        
        

    # Calculam acuratetea
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

    # Se genereaza o permutare noua a tuturor esantioanelor si etichetelor corespunzatoare
    perm = np.random.permutation(train_data.shape[0])
    train_data = train_data[perm,:]
    train_labels = train_labels[perm]

ValueError: cannot reshape array of size 7840000 into shape (60000,784)

In [8]:
# ~ 8. Testarea retelei

# ~ Odata terminata antrenarea retelei, putem testa pe un set de date noi. Veti observa ca structura de la bucla de antrenare ne va ajuta in continuare:

test_data, test_labels = get_MNIST_test()
batch_size_test = 100 # pentru usurinta, ca sa testam toate etichetele alegem un divizor al numarului de date de test
nr_iteratii_test = test_data.shape[0] // batch_size_test
    
predictii = []
for it in range(nr_iteratii_test):
    batch_data = test_data[it*batch_size_test : it*batch_size_test+batch_size_test, :]
    batch_labels = test_labels[it*batch_size_test : it*batch_size_test+batch_size_test]

    current_predict = mlp.forward(batch_data)
    current_predict = np.argmax(current_predict.detach().numpy(),axis=1)
    predictii = np.concatenate((predictii,current_predict))

acc = np.sum(predictii==test_labels)/len(predictii)
print( 'Acuratetea la test este {}%'.format(acc*100) )

Acuratetea la test este 8.55%


In [9]:
# Simplificare

# Modulul nn contine o multitudine de elemente
# esentiale construirii unei retele neuronale
import torch.nn as nn

class Retea_MLP(nn.Module):
    
    def __init__(self, nr_neuroni_input, nr_neuroni_hidden, nr_clase):
        
        # Pentru a putea folosi mai departe reteaua, este recomandata mostenirea
        # clasei de baza nn.Module
        super(Retea_MLP,self).__init__()
        
        self.hidden_layer = nn.Linear(nr_neuroni_input, nr_neuroni_hidden)
        self.out_layer = nn.Linear(nr_neuroni_hidden, nr_clase)

        self.relu = nn.ReLU()
        
    def forward(self,input_batch):
        # Intr-un MLP, intrarea este sub forma unui vector, deci un batch
        # este o matrice de dimensiunea nr_esantioane_batch x dimensiune esantion
        input_batch = torch.from_numpy(input_batch)
        hidden = self.hidden_layer(input_batch)

        self.hidden = self.relu(self.hidden)
        
        out = self.out_layer(hidden)
        
        return out
    
# Instantiem reteaua
mlp = Retea_MLP(28*28,1000,10)

# Specificarea functiei loss
loss_function = nn.CrossEntropyLoss(reduction='sum')

# Specificarea optimizatorului
optim = torch.optim.SGD(mlp.parameters(), lr=1e-5)

In [10]:
train_data, train_labels = get_MNIST_train()
batch_size = 128 # Se poate si mai mult in cazul curent, dar este o valoare frecventa
nr_epoci = 15
nr_iteratii = train_data.shape[0] // batch_size # Din simplitate, vom ignora ultimul batch, daca este incomplet


for ep in range(nr_epoci):
    predictii = []
    etichete = []

    for it in range(nr_iteratii):
        # Luam urmatoarele <batch_size> esantioane si etichete
        batch_data = train_data[it*batch_size : it*batch_size+batch_size, :]
        batch_labels = train_labels[it*batch_size : it*batch_size+batch_size]
        # Se calculeaza predictia retelei pentru datele curente (forward pass/ propagare inainte)
        current_predict = mlp.forward(batch_data)

        # Se calculeaza valoarea momentana a functiei loss
        loss = loss_function(current_predict, torch.from_numpy(batch_labels)) 
        
        # Se memoreaza predictiile si etichetele aferente batch-ului actual (pentru calculul acuratetii)
        current_predict = np.argmax(current_predict.detach().numpy(), axis=1)
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,batch_labels))
        
        # Antrenarea propriu-zisa
        
            # 1. Se sterg toti gradientii calculati anteriori, pentru toate variabilele antrenabile
            # deoarece, metoda <backward> acumuleaza noile valori, in loc sa le inlocuiasca.
        optim.zero_grad()
            # 2. Calculul tuturor gradientilor. Backpropagation
        loss.backward()
            # 3. Actualizarea tuturor ponderilor, pe baza gradientilor.
        optim.step()
        
        

    # Calculam acuratetea
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

    # Se genereaza o permutare noua a tuturor esantioanelor si etichetelor corespunzatoare
    perm = np.random.permutation(train_data.shape[0])
    train_data = train_data[perm,:]
    train_labels = train_labels[perm]

    test_data, test_labels = get_MNIST_test()
    batch_size_test = 100 # pentru usurinta, ca sa testam toate etichetele alegem un divizor al numarului de date de test
    nr_iteratii_test = test_data.shape[0] // batch_size_test
        
    predictii = []
    for it in range(nr_iteratii_test):
        batch_data = test_data[it*batch_size_test : it*batch_size_test+batch_size_test, :]
        batch_labels = test_labels[it*batch_size_test : it*batch_size_test+batch_size_test]

        current_predict = mlp.forward(batch_data)
        current_predict = np.argmax(current_predict.detach().numpy(),axis=1)
        predictii = np.concatenate((predictii,current_predict))

    acc = np.sum(predictii==test_labels)/len(predictii)
    print( 'Acuratetea la test este {}%'.format(acc*100) )

ValueError: cannot reshape array of size 7840000 into shape (60000,784)

In [11]:
torch.save(mlp.state_dict(), './mlp_mnist.pt')

# Descrierea arhitecturii
mlp_antrenat = Retea_MLP(28*28,1000,10)

# Se vor incarca ponderile calculate anterior.
mlp_antrenat.load_state_dict(torch.load('./mlp_mnist.pt'))

# In majoritatea cazurilor, trebuie mentionat faptul ca nu se mai antreneaza reteaua.
# Anumite straturi au comporamente diferite la antrenare, fata de testare. In cazul
# de fata, nu ar trebui sa fie necesara aceasta trecere in modul de inferenta.
mlp_antrenat.eval()

Retea_MLP(
  (hidden_layer): Linear(in_features=784, out_features=1000, bias=True)
  (out_layer): Linear(in_features=1000, out_features=10, bias=True)
  (relu): ReLU()
)

In [13]:
def get_MNIST_train():
    
    mnist_train_data = np.zeros([60000,784])
    mnist_train_labels = np.zeros(60000)
    
    f = open('../../train-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('../../train-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_train_data = np.fromfile(f,dtype=np.uint8).reshape(60000,784)
    mnist_train_labels = np.fromfile(g,dtype=np.uint8)
        
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_train_data = mnist_train_data.astype(np.float32)
    mnist_train_labels = mnist_train_labels.astype(np.int64)
        
    return mnist_train_data, mnist_train_labels

def get_MNIST_test():
    
    mnist_test_data = np.zeros([10000,784])
    mnist_test_labels = np.zeros(10000)
    
    f = open('../../t10k-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('../../t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_test_data = np.fromfile(f,dtype=np.uint8).reshape(10000,784)
    mnist_test_labels = np.fromfile(g,dtype=np.uint8)
    
    # Conversii pentru a se potrivi cu procesul de testare    
    mnist_test_data = mnist_test_data.astype(np.float32)
    mnist_test_labels = mnist_test_labels.astype(np.int64)        
    
    return mnist_test_data, mnist_test_labels



# Modulul nn contine o multitudine de elemente
# esentiale construirii unei retele neuronale
import torch.nn as nn

class Retea_MLP(nn.Module):
    
    def __init__(self, nr_neuroni_input, nr_neuroni_hidden, nr_clase):
        
        # Pentru a putea folosi mai departe reteaua, este recomandata mostenirea
        # clasei de baza nn.Module
        super(Retea_MLP,self).__init__()
        
        self.hidden_layer = nn.Linear(nr_neuroni_input, nr_neuroni_hidden)
        self.relu = nn.ReLU()
        self.out_layer = nn.Linear(nr_neuroni_hidden, nr_clase)
        
    def forward(self,input_batch):
        # Intr-un MLP, intrarea este sub forma unui vector, deci un batch
        # este o matrice de dimensiunea nr_esantioane_batch x dimensiune esantion
        input_batch = torch.from_numpy(input_batch)
        hidden = self.relu(self.hidden_layer(input_batch))
        out = self.out_layer(hidden)
        
        return out
    
# Instantiem reteaua
mlp = Retea_MLP(28*28,1000,10)

# Specificarea functiei loss
loss_function = nn.CrossEntropyLoss(reduction='sum')

# Specificarea optimizatorului
optim = torch.optim.Adam(mlp.parameters(), lr=1e-3)

train_data, train_labels = get_MNIST_train()
batch_size = 128 # Se poate si mai mult in cazul curent, dar este o valoare frecventa
nr_epoci = 15
nr_iteratii = train_data.shape[0] // batch_size # Din simplitate, vom ignora ultimul batch, daca este incomplet


for ep in range(nr_epoci):
    predictii = []
    etichete = []

    for it in range(nr_iteratii):
        # Luam urmatoarele <batch_size> esantioane si etichete
        batch_data = train_data[it*batch_size : it*batch_size+batch_size, :]
        batch_labels = train_labels[it*batch_size : it*batch_size+batch_size]
        # Se calculeaza predictia retelei pentru datele curente (forward pass/ propagare inainte)
        current_predict = mlp.forward(batch_data)

        # Se calculeaza valoarea momentana a functiei loss
        loss = loss_function(current_predict, torch.from_numpy(batch_labels)) 
        
        # Se memoreaza predictiile si etichetele aferente batch-ului actual (pentru calculul acuratetii)
        current_predict = np.argmax(current_predict.detach().numpy(), axis=1)
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,batch_labels))
        
        # Antrenarea propriu-zisa
        
            # 1. Se sterg toti gradientii calculati anteriori, pentru toate variabilele antrenabile
            # deoarece, metoda <backward> acumuleaza noile valori, in loc sa le inlocuiasca.
        optim.zero_grad()
            # 2. Calculul tuturor gradientilor. Backpropagation
        loss.backward()
            # 3. Actualizarea tuturor ponderilor, pe baza gradientilor.
        optim.step()
        
        

    # Calculam acuratetea
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

    # Se genereaza o permutare noua a tuturor esantioanelor si etichetelor corespunzatoare
    perm = np.random.permutation(train_data.shape[0])
    train_data = train_data[perm,:]
    train_labels = train_labels[perm]
    
    
    
test_data, test_labels = get_MNIST_test()
batch_size_test = 100 # pentru usurinta, ca sa testam toate etichetele alegem un divizor al numarului de date de test
nr_iteratii_test = test_data.shape[0] // batch_size_test
    
predictii = []
for it in range(nr_iteratii_test):
    batch_data = test_data[it*batch_size_test : it*batch_size_test+batch_size_test, :]
    batch_labels = test_labels[it*batch_size_test : it*batch_size_test+batch_size_test]

    current_predict = mlp.forward(batch_data)
    current_predict = np.argmax(current_predict.detach().numpy(),axis=1)
    predictii = np.concatenate((predictii,current_predict))

acc = np.sum(predictii==test_labels)/len(predictii)
print( 'Acuratetea la test este {}%'.format(acc*100) )

Acuratetea la epoca 1 este 91.87032585470085%
Acuratetea la epoca 2 este 96.4710202991453%
Acuratetea la epoca 3 este 97.18716613247864%
Acuratetea la epoca 4 este 97.11371527777779%
Acuratetea la epoca 5 este 97.0836672008547%
Acuratetea la epoca 6 este 97.05862713675214%
Acuratetea la epoca 7 este 97.10536858974359%
Acuratetea la epoca 8 este 97.53271901709401%
Acuratetea la epoca 9 este 97.80815972222221%
Acuratetea la epoca 10 este 97.71634615384616%
Acuratetea la epoca 11 este 97.83653846153845%
Acuratetea la epoca 12 este 97.83987713675214%
Acuratetea la epoca 13 este 98.1620592948718%
Acuratetea la epoca 14 este 98.29393696581197%
Acuratetea la epoca 15 este 98.33733974358975%
Acuratetea la test este 96.83%
